In [37]:
!pip install msal
!pip install pandas
!pip install httpx
!pip install pydantic

In [38]:
import requests
from msal import PublicClientApplication
import pandas as pd


app = PublicClientApplication(
    client_id="71055536-ac04-42ad-888a-becd14a863a8",
    authority="https://login.microsoftonline.com/90610670-1a80-434a-88c3-e568bce39bc5",
)
scope = "api://f520238b-3f87-4a44-a4b4-c1424e9f6ea9/access_as_user"
result = app.acquire_token_interactive([scope])
access_token = result['access_token']

site_key = "eng_cf"
date_from = "2023-06-20"
date_to = "2023-06-24"
indicator = "InverterSystemEfficiency"
frequency = "1D"
response = requests.get(
    url=f"https://fd-smart-solar-staging-westus.azurefd.net/Data/{site_key}/inverter",
    params={"from":date_from,"to":date_to, "indicators":indicator, "dataPeriod": frequency},
    headers={
        "Authorization": f"Bearer {access_token}",
        "Accept": "application/json",
    },
)
response.raise_for_status()
response_json = response.json()

# Put response_json into a pandas dataframe
df = pd.DataFrame(response_json)
print(df)

           apiKey                 indicator             timestamp frequency   
0     eng_cf_inv1  InverterSystemEfficiency  2023-06-20T23:00:00Z        1D  \
1     eng_cf_inv1  InverterSystemEfficiency  2023-06-21T23:00:00Z        1D   
2     eng_cf_inv1  InverterSystemEfficiency  2023-06-22T23:00:00Z        1D   
3     eng_cf_inv1  InverterSystemEfficiency  2023-06-23T23:00:00Z        1D   
4   eng_cf_inv1.2  InverterSystemEfficiency  2023-06-20T23:00:00Z        1D   
5   eng_cf_inv1.2  InverterSystemEfficiency  2023-06-21T23:00:00Z        1D   
6   eng_cf_inv1.2  InverterSystemEfficiency  2023-06-22T23:00:00Z        1D   
7   eng_cf_inv1.2  InverterSystemEfficiency  2023-06-23T23:00:00Z        1D   
8     eng_cf_inv2  InverterSystemEfficiency  2023-06-20T23:00:00Z        1D   
9     eng_cf_inv2  InverterSystemEfficiency  2023-06-21T23:00:00Z        1D   
10    eng_cf_inv2  InverterSystemEfficiency  2023-06-22T23:00:00Z        1D   
11    eng_cf_inv2  InverterSystemEfficiency  2023-06

In [39]:
from typing import Any, Union, Optional
import httpx
import datetime  # noqa: F401

import pandas as pd
from pydantic import BaseModel
from msal import PublicClientApplication  # type: ignore

CLIENT_ID = "71055536-ac04-42ad-888a-becd14a863a8"


def dt_str(dt: Union[datetime.datetime , datetime.date]) -> str:
    return dt.strftime("%Y-%m-%dT%H:%M:%SZ")


def str_dt(string: str) -> datetime.datetime:
    return datetime.datetime.strptime(string, "%Y-%m-%dT%H:%M:%SZ")


class Asset(BaseModel):
    asset: str
    type: str
    metadata: Union[dict , None]


class SiteMetaData(BaseModel):
    singletons: dict[str, Union[float , int , datetime.datetime , str , None]]
    monthlyDerateFactorsAc: pd.Series
    monthlyDerateFactorsDc: pd.Series
    owners: list[str]

    class Config:
        arbitrary_types_allowed = True


class Site(BaseModel):
    name: str
    id: str
    timeZone: str
    currency: str
    nativeDataFrequency: str
    metaData: Union[SiteMetaData , None]
    assets: dict[str, Asset]

    def populate_meta_data(self, meta: dict):
        singletons = meta.copy()
        monthlyAc = pd.Series(singletons.pop("monthlyDerateFactorsAc"))
        monthlyDc = pd.Series(singletons.pop("monthlyDerateFactorsDc"))
        owners = singletons.pop("owners")
        self.metaData = SiteMetaData(
            singletons=singletons,
            monthlyDerateFactorsAc=monthlyAc,
            monthlyDerateFactorsDc=monthlyDc,
            owners=owners,
        )


class SolarClient:
    def __init__(self, auth_header: Optional[str] = None, url="https://fd-smart-solar-staging-westus.azurefd.net"):
        self.auth_header = auth_header
        self.base_url = url

    @classmethod
    def from_interactive_auth(
            cls,
    ) -> 'SolarClient':
        auth_url="https://login.microsoftonline.com/90610670-1a80-434a-88c3-e568bce39bc5"
        client = PublicClientApplication(client_id=CLIENT_ID, authority=auth_url)
        token = client.acquire_token_interactive(
            scopes=["api://f520238b-3f87-4a44-a4b4-c1424e9f6ea9/access_as_user"]
        )
        if "error" in token.keys():
            raise Exception("Error in authenticating :(")
        return SolarClient(auth_header= "Bearer " + token["access_token"])

    def get(self, url: str, params: dict[str, Any]) -> dict[str, str]:
          response = httpx.get(
            url=f"{self.base_url}/{url}",
            params=params,
            headers={"Authorization": self.auth_header, "Accept": "application/json"},
        )

    def get_sites(self) -> dict[str, str]:
        response = httpx.get(
            url=f"{self.base_url}/Sites",
            headers={"Authorization": self.auth_header, "Accept": "application/json"},
        )
        response.raise_for_status()
        site_list = response.json()["data"]
        return_val = {}
        for site in site_list:
            return_val[site["name"]] = site["id"]

        return return_val

    def get_site_info(self, site_id: str) -> Site:
        response = httpx.get(
            url=f"{self.base_url}/Sites/{site_id}",
            headers={"Authorization": self.auth_header, "Accept": "application/json"},
        )
        response.raise_for_status()
        data = response.json()["data"]

        assets = {}
        for x in data["assets"]:
            assets[x["resAssetCode"]] = Asset(
                asset=x["resAssetCode"], type=x["assetType"], metadata=x["metadata"]
            )

        site = Site(
            name=data["name"],
            id=data["id"],
            timeZone=data["timeZone"],
            currency=data["currency"],
            nativeDataFrequency=data["nativeDataFrequency"],
            assets=assets,
            metaData=None,
        )
        site.populate_meta_data(data["siteMetaData"])
        return site

    def get_data(
            self,
            site_id: str,
            asset_type: str,
            indicator: str,
            start: datetime.datetime,
            end: datetime.datetime,
            period="",
    ) -> pd.DataFrame:
        response = httpx.get(
            url=f"{self.base_url}/Data/{site_id}/{asset_type}",
            headers={
                "Authorization": self.auth_header,
            },
            params={
                "indicators": [indicator],
                "from": dt_str(start),
                "to": dt_str(end),
                "period": period,
            },
        )
        cleaned = response.json()
        for x in cleaned:
            print(x)
            x["timestamp"] = str_dt(x["timestamp"])
        result = pd.DataFrame(cleaned)
        return result.set_index("timestamp")

    def list_asset_types(self, site_id: str) -> set[str]:
        response = httpx.get(
            url=f"{self.base_url}/Sites/{site_id}/assetTypes",
            headers={"Authorization": self.auth_header, "Accept": "application/json"},
        )
        return set(response.json())

    def list_indicators(self, site_id: str) -> dict[str, pd.DataFrame]:
        response = httpx.get(
            url=f"{self.base_url}/Sites/{site_id}/indicators",
            headers={"Authorization": self.auth_header, "Accept": "application/json"},
        )
        assets = response.json()
        assets = {
            asset: pd.DataFrame(
                [{"name": x["name"], "unit": x["unit"]} for x in assets[asset]]
            ).set_index("name")
            for asset in assets
        }
        return assets

    def list_indicators_per_asset_type(self, site_id: str, asset_type: str) -> pd.DataFrame:
        response = httpx.get(
            url=f"{self.base_url}/Sites/{site_id}/indicators/assetTypes/{asset_type}",
            headers={"Authorization": self.auth_header, "Accept": "application/json"},
        )
        return pd.DataFrame(
            [{"name": x["name"], "unit": x["unit"]} for x in response.json()]
        ).set_index("name")

    def list_assets(self, site_id: str, asset_type: str) -> set[str]:
        response = httpx.get(
            url=f"{self.base_url}/Sites/site/{site_id}/assetType/{asset_type}",
            headers={"Authorization": self.auth_header, "Accept": "application/json"},
        )
        return set(response.json()["data"])
solar_client = SolarClient.from_interactive_auth()

In [40]:
def calculate_mse(series_a, series_b):
    """
    Calculates the mean squared error (MSE) between two pandas Series.

    Parameters:
        series_a (pd.Series): First time-series pandas Series.
        series_b (pd.Series): Second time-series pandas Series.

    Returns:
        float: The mean squared error between the two Series.
    """
    if not isinstance(series_a, pd.Series) or not isinstance(series_b, pd.Series):
        raise ValueError("Both inputs must be pandas Series.")

    if len(series_a) != len(series_b):
        raise ValueError("Both Series must have the same length.")

    squared_errors = (series_a - series_b) ** 2
    mse = squared_errors.mean()
    return mse

# Example usage with time-series data:
# Replace the data in the Series with your own data
timestamps = pd.date_range(start='2023-07-20', periods=5, freq='D')
values_a = [1.0, 2.0, 3.0, 4.0, 5.0]
values_b = [1.2, 1.8, 3.2, 4.5, 5.5]

series_a = pd.Series(values_a, index=timestamps)
series_b = pd.Series(values_b, index=timestamps)

mse = calculate_mse(series_a, series_b)
print("Mean Squared Error:", mse)

Mean Squared Error: 0.124


In [41]:
site_metadata = solar_client.get_site_info("eng_cf")
inv_data = solar_client.get_data("eng_cf", asset_type="inverter", indicator="InverterSystemEfficiency", start=datetime.datetime.fromisoformat("2023-03-01"),end=datetime.datetime.fromisoformat("2023-03-02"), period = '1D')
print(site_metadata)
print(inv_data)

{'apiKey': 'eng_cf_inv1', 'indicator': 'InverterSystemEfficiency', 'timestamp': '2023-03-01T00:00:00Z', 'frequency': '15min', 'value': None, 'unit': ''}
{'apiKey': 'eng_cf_inv1', 'indicator': 'InverterSystemEfficiency', 'timestamp': '2023-03-01T00:15:00Z', 'frequency': '15min', 'value': None, 'unit': ''}
{'apiKey': 'eng_cf_inv1', 'indicator': 'InverterSystemEfficiency', 'timestamp': '2023-03-01T00:30:00Z', 'frequency': '15min', 'value': None, 'unit': ''}
{'apiKey': 'eng_cf_inv1', 'indicator': 'InverterSystemEfficiency', 'timestamp': '2023-03-01T00:45:00Z', 'frequency': '15min', 'value': None, 'unit': ''}
{'apiKey': 'eng_cf_inv1', 'indicator': 'InverterSystemEfficiency', 'timestamp': '2023-03-01T01:00:00Z', 'frequency': '15min', 'value': None, 'unit': ''}
{'apiKey': 'eng_cf_inv1', 'indicator': 'InverterSystemEfficiency', 'timestamp': '2023-03-01T01:15:00Z', 'frequency': '15min', 'value': None, 'unit': ''}
{'apiKey': 'eng_cf_inv1', 'indicator': 'InverterSystemEfficiency', 'timestamp': '2

In [42]:
# Example model:


dc_power_inv  = solar_client.get_data("eng_cf", asset_type="inverter", indicator="ActivePowerDC_Mean", start=datetime.datetime.fromisoformat("2023-03-01"),end=datetime.datetime.fromisoformat("2023-03-02"))

satellite_data = solar_client.get_data("eng_cf", asset_type="site", indicator="FrontsidePOA_Mean", start=datetime.datetime.fromisoformat("2023-03-01"),end=datetime.datetime.fromisoformat("2023-03-02"), period="5min")

{'apiKey': 'eng_cf_inv1', 'indicator': 'ActivePowerDC_Mean', 'timestamp': '2023-03-01T00:00:00Z', 'frequency': '15min', 'value': 0, 'unit': 'kW'}
{'apiKey': 'eng_cf_inv1', 'indicator': 'ActivePowerDC_Mean', 'timestamp': '2023-03-01T00:15:00Z', 'frequency': '15min', 'value': 0, 'unit': 'kW'}
{'apiKey': 'eng_cf_inv1', 'indicator': 'ActivePowerDC_Mean', 'timestamp': '2023-03-01T00:30:00Z', 'frequency': '15min', 'value': 0, 'unit': 'kW'}
{'apiKey': 'eng_cf_inv1', 'indicator': 'ActivePowerDC_Mean', 'timestamp': '2023-03-01T00:45:00Z', 'frequency': '15min', 'value': 0, 'unit': 'kW'}
{'apiKey': 'eng_cf_inv1', 'indicator': 'ActivePowerDC_Mean', 'timestamp': '2023-03-01T01:00:00Z', 'frequency': '15min', 'value': 0, 'unit': 'kW'}
{'apiKey': 'eng_cf_inv1', 'indicator': 'ActivePowerDC_Mean', 'timestamp': '2023-03-01T01:15:00Z', 'frequency': '15min', 'value': 0, 'unit': 'kW'}
{'apiKey': 'eng_cf_inv1', 'indicator': 'ActivePowerDC_Mean', 'timestamp': '2023-03-01T01:30:00Z', 'frequency': '15min', 'val

In [43]:
dc_power_inv

,apiKey,indicator,frequency,value,unit
timestamp,,,,,
2023-03-01 00:00:00,eng_cf_inv1,ActivePowerDC_Mean,15min,0.0,kW
2023-03-01 00:15:00,eng_cf_inv1,ActivePowerDC_Mean,15min,0.0,kW
2023-03-01 00:30:00,eng_cf_inv1,ActivePowerDC_Mean,15min,0.0,kW
2023-03-01 00:45:00,eng_cf_inv1,ActivePowerDC_Mean,15min,0.0,kW
2023-03-01 01:00:00,eng_cf_inv1,ActivePowerDC_Mean,15min,0.0,kW
...,...,...,...,...,...
2023-03-01 22:45:00,eng_cf_inv4.2,ActivePowerDC_Mean,15min,0.0,kW
2023-03-01 23:00:00,eng_cf_inv4.2,ActivePowerDC_Mean,15min,0.0,kW
2023-03-01 23:15:00,eng_cf_inv4.2,ActivePowerDC_Mean,15min,0.0,kW


In [44]:
df_inv_1 = dc_power_inv[dc_power_inv['apiKey'] == 'eng_cf_inv1']

In [45]:
df_inv_1_series = df_inv_1["value"]
satellite_data_series = satellite_data["value"]

In [46]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression


# Convert Series to a 2D numpy array (required by scikit-learn)
X = df_inv_1_series.values.reshape(-1, 1)  # Reshape to (n_samples, n_features)
y = satellite_data_series.values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and fit the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Print the coefficients and intercept of the linear regression model
print("Coefficient:", model.coef_[0])
print("Intercept:", model.intercept_)

Coefficient: 1.245328404867998
Intercept: 9.714966521559866


In [47]:
# Predict the values for the test set
y_pred = model.predict(X_test)

# Optionally, you can calculate the mean squared error (MSE) on the test set
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 663.3688107617439
